In [1]:
#imports
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import torch

from fastai.conv_learner import *
from fastai.dataset import *

from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects

import multiprocessing
from functools import partial
import shutil
import itertools

from sklearn.metrics import roc_auc_score

In [2]:
from torchvision import datasets, models, transforms,utils

In [3]:
cuda.set_device(1)

In [4]:
def auc_avg(pred, target):
    '''
    Computes mean ROC-AUC by averaging AUC for each class
    args:
    pred: torch tensor, probability prediction from the network (batch size x n_classes)
    target: torch tensor, True binary labels (batch size x n_classes)
    
    output:
    average AUC score
    '''
    aucs = []
    n_class = target.size()[1]
    target_np = target.cpu().numpy()
    pred_np = pred.cpu().numpy()
    for i in range(n_class):
        if target_np[:, i].sum() > 0:
            aucs.append(roc_auc_score(target_np[:, i], pred_np[:, i]))
    return np.mean(aucs)

In [5]:
def compute_aucs(pred, target):
    '''
    Computes AUC for each class
    args:
    pred: numpy array, probability prediction from the network (batch size x n_classes)
    target: numpy array, True binary labels (batch size x n_classes)
    
    output:
    List of AUC for each class
    '''
    aucs = []
    n_class = target.shape[1]
    for i in range(n_class):
        if target[:, i].sum() > 0:
            aucs.append(roc_auc_score(target[:, i], pred[:, i]))
    return aucs

In [6]:
def bulk_copyfiles(filelist, source, destination, overwrite = True):
    '''
    filelist: list of filenames you need to copy
    source: source directory
    destination: destination directory
    '''
    if os.path.exists(destination) and overwrite: shutil.rmtree(destination)
    os.makedirs(destination, exist_ok=True)
    for fname in filelist:
        if os.path.exists(source + fname):
            shutil.copy(os.path.join(source, fname), destination)

In [7]:
trn_list = pd.read_csv('data/labels/train_list.txt',header=None)

In [8]:
trn_imgs = list(trn_list[0].apply(lambda x: x.split(' ')[0]))

In [9]:
len(trn_imgs)

78468

In [10]:
val_list = pd.read_csv('data/labels/val_list.txt',header=None)

val_list.head()

,0
0,00028178_000.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
1,00030605_000.png 0 0 0 1 0 0 0 0 0 0 0 0 0 0
2,00011297_000.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
3,00011297_001.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
4,00016540_000.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [11]:
val_imgs = list(val_list[0].apply(lambda x: x.split(' ')[0]))

In [12]:
len(val_imgs)

11219

In [13]:
test_list = pd.read_csv('data/labels/test_list.txt',header=None)

test_list.head()

,0
0,00011997_000.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
1,00011997_001.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
2,00011997_002.png 0 0 0 0 0 0 0 0 0 0 0 0 0 0
3,00011997_003.png 0 0 1 0 0 0 0 0 0 0 0 0 0 0
4,00011997_004.png 0 1 1 1 0 0 0 0 0 0 0 0 0 0


In [14]:
test_imgs = list(test_list[0].apply(lambda x: x.split(' ')[0]))

In [15]:
len(test_imgs)

22433

In [16]:
path = 'data/resnet50_ft/'

label_csv = f'{path}labels_ft.csv'

label_ft = pd.read_csv(label_csv) #label for the full dataset

In [17]:
#creating train, validation and test idxs
trn_idx = np.array(label_ft.index[label_ft.img_id.isin(trn_imgs)])

val_idx = np.array(label_ft.index[label_ft.img_id.isin(val_imgs)])

tst_idx = np.array(label_ft.index[label_ft.img_id.isin(test_imgs)])

In [18]:
# set(val_idx).intersection(set(tst_idx)) 
#checking that no ID is common among train, val and test. No ID common

In [19]:
# #copying test images to a separate folder (done)
# bulk_copyfiles(
#     test_imgs,
#     'data/nih_resized/',
#     'data/resnet_ft/test_imgs/',
#     overwrite=True)  

### Resnext


In [20]:
#augmentation
aug_tfms = [
    RandomRotate(3, p=0.5),
    RandomLighting(0.05, 0.05),
]

In [21]:
arch = resnext_101_64x4d

sz = 300

bs = 64

In [22]:
tfms = tfms_from_model(arch,sz,aug_tfms=aug_tfms,max_zoom=1.1,)

In [23]:
data = ImageClassifierData.from_csv(
    'data/',
    'nih_resized',
    csv_fname=label_csv,
    val_idxs=val_idx,
    tfms=tfms,
    bs=bs,
    test_name='resnet50_ft/test_imgs/')

In [24]:
base_model = resnext_101_64x4d().cuda()

In [25]:
weights = torch.load("./fastai/weights/resnext_101_64x4d.pth")

In [26]:
base_model.load_state_dict(weights)

In [27]:
c = children(base_model)

In [28]:
len(c)

11

In [29]:
c[-1] #last layer

Sequential(
  (0): Lambda(
  )
  (1): Linear(in_features=2048, out_features=1000, bias=True)
)

Need to change the last layer to output 15 class prediction

In [30]:
model_new = nn.Sequential(*c[:-1]+[nn.Linear(2048,15)]+[nn.Sigmoid()]).cuda()

In [31]:
children(model_new)[-2:]

[Linear(in_features=2048, out_features=15, bias=True), Sigmoid()]

In [32]:
inp = V(torch.ones(1,3,224,224))

In [33]:
model_new(inp) #checking with a sample input

Variable containing:

Columns 0 to 9 
 0.4929  0.5358  0.4816  0.4391  0.4949  0.5766  0.4907  0.5987  0.3905  0.3963

Columns 10 to 14 
 0.5768  0.5399  0.4738  0.6195  0.4773
[torch.cuda.FloatTensor of size 1x15 (GPU 1)]

In [34]:
metric = [auc_avg] #validation metric to be reported while training

In [35]:
class ResnextModel():
    def __init__(self, model, cut_lr):
        self.model, self.cut_lr = model, cut_lr
    def get_layer_groups(self, precompute):
        lgs = list(split_by_idxs(children(self.model), self.cut_lr))
        return lgs

In [36]:
m = ResnextModel(model_new, [5,9])

In [38]:
learn = ConvLearner(data, m)

In [39]:
lrf = learn.lr_find()

  0%|          | 0/1577 [00:00<?, ?it/s]


RuntimeError: size mismatch at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCTensorMathBlas.cu:247